In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import string
from tqdm import tqdm

# Define the model
class GRUTextGenerator(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(GRUTextGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        # x: (batch_size, seq_length)
        embedded = self.embedding(x)  # (batch_size, seq_length, embed_size)
        output, hidden = self.gru(embedded, hidden)  # (batch_size, seq_length, hidden_size)
        logits = self.fc(output)  # (batch_size, seq_length, vocab_size)
        return logits, hidden

token_dict = {letter:i for letter,i in zip(string.ascii_lowercase, range(1,27))}
token_dict[' '] = 0

class TextDataset(Dataset):
    def __init__(self, sentences, idx):
        self.data = []

        self.max_len = max([len(x) for x in sentences])

        for sentence in sentences:
            if len(sentence) > idx:
                x = torch.tensor([token_dict[x] for x in sentence[:idx]], dtype=torch.int64, device=device)
                y = torch.tensor(token_dict[sentence[idx]], dtype=torch.int64, device=device)
                self.data.append((x, y))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]

# Training loop
def train(model, dataloaders, vocab_size, device, epochs=10, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.to(device)
    model.train()
    
    for epoch in range(epochs):
        for data_loader in dataloaders:
            total_loss = 0
            for _, (inputs, targets) in enumerate(data_loader, 0):
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                
                logits, _ = model(inputs)  # (batch_size, seq_length, vocab_size)
                logits = logits[:, -1, :]
                logits = logits.view(-1, vocab_size)  # Reshape for loss calculation
                targets = targets.view(-1)  # Reshape to match logits
                
                loss = criterion(logits, targets)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(data_loader)}")

# Example hyperparameters
vocab_size = 27  # a-z and a space character
embed_size = 128
hidden_size = 256
num_layers = 2
batch_size = 64
seq_length = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create the model
model = GRUTextGenerator(vocab_size, embed_size, hidden_size, num_layers)



In [ ]:
# Placeholder: Replace with your own DataLoader
sentences = []
with open("simple_sentences.txt", 'r') as f:
    sentences = f.readlines()
sentences = [x.strip() for x in sentences]

dataloaders = []
for i in range(1, max([len(x) for x in sentences])):
    dataset = TextDataset(sentences, i)
    dataloaders.append(DataLoader(dataset, batch_size))

# Example training call
train(model, dataloaders, vocab_size, device)

In [93]:
def vectorise(sentence):
    return torch.tensor([token_dict[x] for x in sentence], dtype=torch.int64, device=device)

def run_inference(model, sentence):
    input = vectorise(sentence)

    output = model(input)

    logits = output[0][-1,:]

    max_idx = logits.argmax().item()
    for a, i in token_dict.items():
        if max_idx == i:
            sentence += a
            return sentence
    
    print(max_idx)



sentence = 'a dog run'


In [94]:
for i in range(12):
    sentence = run_inference(model, sentence)
    print(sentence)

a dog runs
a dog runs 
a dog runs o
a dog runs on
a dog runs on 
a dog runs on t
a dog runs on th
a dog runs on the
a dog runs on the 
a dog runs on the b
a dog runs on the be
a dog runs on the bed


In [ ]:
import numpy as np
import json
from json import JSONEncoder


In [105]:
class EncodeTensor(JSONEncoder,Dataset):
    def default(self, obj):
        if isinstance(obj, torch.Tensor):
            return obj.cpu().detach().numpy().tolist()
        return super(json.NpEncoder, self).default(obj)


In [106]:
x = vectorise(sentence)
y = model(x)[0]

np.savetxt('test_data/generative_gru_x.csv', x.detach(), delimiter=',')
np.savetxt('test_data/generative_gru_y.csv', y.detach(), delimiter=',')

with open('models/generative_gru.json', 'w') as json_file:
    json.dump(model.state_dict(), json_file,cls=EncodeTensor)
